In [1]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("apjspeech.pdf")
docs = loader.load_and_split()
docs

[Document(metadata={'source': 'apjspeech.pdf', 'page': 0}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those livi ng abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealth of our country. During my intera ction at \nRashtrapati Bhavan in Delhi and at every state and union territor y as well as through my \nonline interactions, I have many unique experiences to share with you, which signify the \nfollowi

In [2]:
from langchain_groq import ChatGroq
import os
api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(groq_api_key=api_key, model="Gemma-7b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001E99BA65370>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001E99B9D7E00>, model_name='Gemma-7b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain.schema import(
    AIMessage,
    HumanMessage, SystemMessage
)
from langchain.chains import LLMChain
from langchain import PromptTemplate

In [4]:
template="""
Write a concise and short summary of the following speech,
Speech: {text}

"""
prompt=PromptTemplate(input_variables=['text'], template=template)

In [5]:
from langchain.chains.summarize import load_summarize_chain

In [6]:
chain=load_summarize_chain(llm, chain_type='stuff', prompt=prompt, verbose=True)
output_summary=chain.run(docs)
output_summary

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_19320\2812730720.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output_summary=chain.run(docs)




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Write a concise and short summary of the following speech,
Speech: A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those livi ng abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online in

'**Summary: A P J Abdul Kalam\'s Departing Speech**\n\nFormer President of India, Dr. A. P. J. Abdul Kalam, delivered a heartfelt departing speech, reflecting on his five-year tenure in office. He shared inspiring stories and initiatives that highlighted the transformative power of people and technology in achieving national development.\n\n**Key Points:**\n\n**1. Empowering Villages and Rural Communities:**\n- Emphasis on empowering rural communities through PURA (Providing Urban Amenities in Rural Areas) initiative.\n- Role of technology for connectivity, education, and healthcare in rural areas.\n\n\n**2. Agro-Economic Growth and Sustainability:**\n- Emphasis on strengthening agriculture through "Seed to Food" initiative.\n- Encouraging farmers and promoting value addition to boost agricultural GDP growth.\n\n\n**3. Overcoming Challenges and Disasters:**\n- Inspiring stories of resilience and courage during natural disasters like earthquakes and tsunamis.\n- Highlight of the Car Nic

<h1 style="color:red">Map reduce to summarize Large documents<h1>

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [8]:
docs

[Document(metadata={'source': 'apjspeech.pdf', 'page': 0}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those livi ng abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealth of our country. During my intera ction at \nRashtrapati Bhavan in Delhi and at every state and union territor y as well as through my \nonline interactions, I have many unique experiences to share with you, which signify the \nfollowi

In [9]:
final_document = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100).split_documents(documents=docs)
final_document

[Document(metadata={'source': 'apjspeech.pdf', 'page': 0}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those livi ng abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealth of our country. During my intera ction at \nRashtrapati Bhavan in Delhi and at every state and union territor y as well as through my \nonline interactions, I have many unique experiences to share with you, which signify the \nfollowi

In [10]:
len(final_document)

16

In [11]:
prompt

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\nWrite a concise and short summary of the following speech,\nSpeech: {text}\n\n')

In [12]:
## create a new prompt
chunks_prompt="""
Please summarize the below speech:
Speech: {text}
summary:
"""

map_prompt_template=PromptTemplate(input_variables=['text'], template=chunks_prompt)

In [14]:
final_prompt = """
Provide the final summary of the entire speech with these important points.
Add a Motivation Title, Start the precise summary with an introduction and provide e4the summary in number point for the speech.
Speech:{text}
    """
final_prompt_template = PromptTemplate(
    input_variables=['text'], 
    template=final_prompt
    
)
final_prompt_template

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\nProvide the final summary of the entire speech with these important points.\nAdd a Motivation Title, Start the precise summary with an introduction and provide e4the summary in number point for the speech.\nSpeech:{text}\n    ')

In [15]:
summary_chain=load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",
    map_prompt=map_prompt_template,
    combine_prompt=final_prompt_template,
    verbose=True,
)

output=summary_chain.run(final_document)
output



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Please summarize the below speech:
Speech: A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those livi ng abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online interactions, I have m

c:\Users\Lenovo\anaconda3\envs\torchGPU\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Token indices sequence length is longer than the specified maximum sequence length for this model (1854 > 1024). Running this sequence through the model will result in indexing errors




> Entering new LLMChain chain...
Prompt after formatting:

Provide the final summary of the entire speech with these important points.
Add a Motivation Title, Start the precise summary with an introduction and provide e4the summary in number point for the speech.
Speech:**Summary:**

Former President of India, A.P.J. Abdul Kalam, expresses his heartfelt gratitude to the people of India for the privilege of serving as their President for five remarkable years. He highlights the importance of empowering the youth, accelerating development, and mobilizing rural core competence for competitiveness. He emphasizes the need for a strong agricultural sector through initiatives like "Seed to Food" to ensure food security. Kalam also stresses the importance of tackling problems through partnerships, exhibiting courage in the face of calamities, and promoting connectivity for societal transformation. He concludes by urging the youth to embrace the "Youth Movement for Developed India 2020" to ac

'## Speech Summary: A Vision for Developed India 2020\n\n**Introduction:**\n\nFormer President A.P.J. Abdul Kalam expresses heartfelt gratitude for the opportunity to serve India and outlines his vision for a developed nation by 2020.\n\n**Key Points:**\n\n**1. Empowering the Youth:**\n\n- Importance of empowering the youth through education, especially in rural areas.\n- Emphasis on the "Youth Movement for Developed India 2020" to achieve national aspirations.\n\n**2. Sustainable Development:**\n\n- Need for balanced development that preserves cultural heritage and natural resources.\n- Confidence in achieving economic growth through agricultural, industrial, and service sector collaboration.\n\n**3. Social Transformation:**\n\n- Emphasis on inclusive education and eradication of poverty, illiteracy, and gender-based violence.\n- Vision of universal access to healthcare and responsive governance.\n\n**4. National Progress:**\n\n- Highlights India\'s achievements in the past 60 years.\